In [ ]:
# from _typeshed import IdentityFunction
import numpy as np
from copy import deepcopy
from operator import attrgetter
np.random.seed(0)

class Solution:
    def __init__(self, pixels, values, x, p_size):
        self.pixels = pixels  # list of Integers
        self.values = values  # list of Binary tuples, i.e. [0, 1, 1]
        self.x = x  # (w x w x 3)
        self.fitnesses = []
        self.is_adversarial = None
        self.w = x.shape[0]
        self.delta = len(self.pixels)
        self.domination_count = None
        self.dominated_solutions = None
        self.rank = None
        self.crowding_distance = None

        self.loss = None
        self.p_size = p_size

    def copy(self):
        return deepcopy(self)

    def generate_image(self):
        x_adv = self.x.copy()
        for a in range(self.delta):
            x_adv[self.pixels[a] // self.w, self.pixels[a] % self.w] += (self.values[a] * self.p_size)

        return np.clip(x_adv, 0, 1)

    def evaluate(self, loss_function):
        img_adv = self.generate_image()
        fs = loss_function(img_adv)
        self.is_adversarial = fs[0]  # Assume first element is boolean always

        # the fitness function will follow the format of [loss, l_0_norm, l_infinity_norm]
        self.fitnesses = fs[1:] # Loss

        l_0 = np.count_nonzero(np.sum(abs(self.values),axis=1))
        self.fitnesses.append(l_0)

        # l_infinity = np.max(img_adv)
        self.fitnesses.append(self.p_size)

        self.fitnesses = np.array(self.fitnesses)

        self.loss = fs[1]

    def dominates(self, soln):
        if self.is_adversarial is True and soln.is_adversarial is False:
            return True

        if self.is_adversarial is False and soln.is_adversarial is True:
            return False

        if self.is_adversarial is True and soln.is_adversarial is True:
          if self.fitnesses[1] < soln.fitnesses[1] and not self.fitnesses[2] > soln.fitnesses[2]:
              return True
          elif self.fitnesses[2] < soln.fitnesses[2] and not self.fitnesses[1] > soln.fitnesses[1]:
              return True

        if self.is_adversarial is False and soln.is_adversarial is False:
            return True if self.fitnesses[0] < soln.fitnesses[0] else False


def fast_nondominated_sort(population):
    fronts = [[]]
    for individual in population:
        individual.domination_count = 0
        individual.dominated_solutions = []
        for other_individual in population:
            if individual.dominates(other_individual):
                individual.dominated_solutions.append(other_individual)
            elif other_individual.dominates(individual):
                individual.domination_count += 1
        if individual.domination_count == 0:
            individual.rank = 0
            fronts[0].append(individual)
    i = 0
    while len(fronts[i]) > 0:
        temp = []
        for individual in fronts[i]:
            for other_individual in individual.dominated_solutions:
                other_individual.domination_count -= 1
                if other_individual.domination_count == 0:
                    other_individual.rank = i + 1
                    temp.append(other_individual)
        i = i + 1
        fronts.append(temp)

    return fronts


def calculate_crowding_distance(front):
    if len(front) > 0:
        solutions_num = len(front)
        for individual in front:
            individual.crowding_distance = 0

        for m in range(len(front[0].fitnesses)):
            front.sort(key=lambda individual: individual.fitnesses[m])
            front[0].crowding_distance = 10 ** 9
            front[solutions_num - 1].crowding_distance = 10 ** 9
            m_values = [individual.fitnesses[m] for individual in front]
            scale = max(m_values) - min(m_values)
            if scale == 0: scale = 1
            for b in range(1, solutions_num - 1):
                front[b].crowding_distance += (front[b + 1].fitnesses[m] - front[b - 1].fitnesses[m]) / scale


def crowding_operator(individual, other_individual):
    if (individual.rank < other_individual.rank) or ((individual.rank == other_individual.rank) and (
            individual.crowding_distance > other_individual.crowding_distance)):
        return 1
    else:
        return -1


def __tournament(population, tournament_size):
    participants = np.random.choice(population, size=(tournament_size,), replace=False)
    best = None
    for participant in participants:
        if best is None or (crowding_operator(participant, best) == 1):
            best = participant

    return best


def tournament_selection(population, tournament_size):
    parents = []
    while len(parents) < len(population) // 2:
        # print("len_parents",len(parents))
        # print("len_population",len(population))
        parent1 = __tournament(population, tournament_size)
        temp_pop = population.copy()
        temp_pop.remove(parent1)
        parent2 = __tournament(temp_pop, tournament_size)
        # print("bam, parents created")

        # while np.array_equal(temp1.pixels,temp2.pixels):
        #   temp2 = __tournament(population, tournament_size).copy()
        # print("temp2 is verified to be different from temp1")

        # parent1 = temp1.copy()
        # parent2 = temp2.copy()
        # print("parents created")

        parents.append([parent1, parent2])
    return parents